# Visualizing Optimization History in Jupyter Notebook

This notebook demonstrates a visualization utility of Optuna.
After optimizing the hyperparameter of neural networks, `plot_optimization_history()` plots optimization history in a study.

## Setting up MNIST Dataset

In [1]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

mnist = fetch_openml('mnist_784', version=1)
classes = list(set(mnist.target))
x_train, x_test, y_train, y_test = train_test_split(mnist.data, mnist.target)

## Defining Objective Function

In [2]:
from sklearn.neural_network import MLPClassifier

def objective(trial):

    layers = []
    n_layers = trial.suggest_int('n_layers', 1, 4)
    for i in range(n_layers):
        layers.append(trial.suggest_int('n_units_l{}'.format(i), 1, 128))

    clf = MLPClassifier(hidden_layer_sizes=tuple(layers))

    for step in range(100):
        clf.partial_fit(x_train, y_train, classes=classes)

        intermediate_value = 1.0 - clf.score(x_test, y_test)  # Report intermediate objective value.
        trial.report(intermediate_value, step)

        if trial.should_prune(step):
            raise optuna.structs.TrialPruned()  # Handle pruning based on the intermediate value.

    return 1.0 - clf.score(x_test, y_test)

## Running Optimization

In [3]:
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)  # This verbosity change is just to simplify the notebook output.

# study = optuna.create_study(pruner=optuna.pruners.SuccessiveHalvingPruner())
study = optuna.create_study()
study.optimize(objective, n_trials=100)

## Plotting Optimization History of Study

In [4]:
import optuna.visualization as visualization

In [5]:
visualization.plot_intermediate_values(study)

In [6]:
import optuna.dashboard as dashboard

In [7]:
dashboard.write(study, "output.html")

[W 2019-09-20 21:07:35,411] Optuna dashboard is still highly experimental. Please use with caution!


In [ ]:
trials = study.trials_dataframe()
trials.head()

In [ ]:
!pip install pandas-profiling

In [ ]:
import pandas_profiling